<a href="https://colab.research.google.com/github/himanshugaur17/spend-classifier/blob/feature%2Finitial-setup/data_initial_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
stmnt_df=pd.read_csv('/content/drive/MyDrive/Github/spend-classifier/feb-stmnt.csv')

Mounted at /content/drive


Stripping columns names of leading and trailing whitespaces as:


---


```
map(lambda x:x.strip(),stmnt_df.columns.tolist())

```



In [45]:
stmnt_df.columns=list(map(lambda x:x.strip(),stmnt_df.columns.tolist()))

In [29]:
def filterOutNoise(regex_filter,df,column):
  filtered_out_df=df[df[column].str.contains(regex_filter,regex=True)]
  return filtered_out_df.reset_index()

The below applyFunc gets called two times, one for Debit series and second time for the Credit series.
This function is given as parameter pd.apply(here passed as input)

---

```
daily_debit_txns_df[['Debit','Credit']].apply(myApplyFunc)
```


In [30]:
def myApplyFunc(x):
  return pd.to_numeric(x.str.replace(',',''),errors='coerce')

In [49]:
regex_to_take_out_debit_txns='TO TRANSFER|DEBIT(?! SWEEP)'
daily_debit_txns_df=filterOutNoise(regex_to_take_out_debit_txns,stmnt_df,'Description')
daily_debit_txns_df[['Debit','Credit']]=daily_debit_txns_df[['Debit','Credit']].apply(myApplyFunc)
daily_debit_txns_df[daily_debit_txns_df['Debit']==daily_debit_txns_df['Debit'].max()]
total_debit=daily_debit_txns_df['Debit'].sum()
invstmnts_df=daily_debit_txns_df[daily_debit_txns_df['Description'].str.contains('DEBIT|Indian',regex=True)]
debit_df=pd.concat([invstmnts_df,daily_debit_txns_df]).drop_duplicates(ignore_index=True,keep=False)
monthly_invstment_tot=invstmnts_df['Debit'].sum()

42961.99
